# fraud in Credit Card

## DataFrame Description:
* ## Time: Timestamp of the transaction.
* ## V1-V28: Features generated by PCA transformation to protect user identities and sensitive information.
* ## Amount: Transaction amount.
* ## Class: Class label indicating whether the transaction is fraudulent (1) or not (0).

In [55]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [56]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [57]:
# first 5 rows of the dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [58]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [59]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [60]:
credit_card_data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [61]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [62]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [63]:
# statistical measures of the data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [64]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [65]:
# compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 492

In [66]:
legit_sample = legit.sample(n=492)

In [67]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [68]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
147509,88628.0,1.720751,-0.726169,-0.808672,0.279894,0.227486,1.423688,-0.622403,0.436983,1.080781,-0.210653,1.046919,1.741840,0.587748,-0.042244,-0.093882,-1.009232,0.230936,-0.923577,-0.772597,-0.157154,0.354501,1.339983,0.006461,-1.346720,-0.088529,-0.019991,0.070680,-0.059333,68.25,0
51130,44808.0,-7.390234,-2.362138,-1.601340,2.305168,1.322983,0.388169,-1.871498,-5.730326,-0.277777,0.920010,1.855357,1.344497,-0.163115,0.995140,1.300396,-0.312151,0.379460,-0.736587,0.419206,-1.389719,-3.144408,1.454596,-0.719147,-0.099626,-0.343214,-0.397844,-2.403415,1.671146,5.92,0
259545,159165.0,-0.704958,0.536297,1.287091,-0.470781,0.451874,0.735918,0.037005,0.310093,-0.080568,-1.080879,0.726941,0.597830,1.072433,-1.784209,0.210283,1.232057,-0.048924,1.383463,0.459825,0.175846,-0.047403,-0.157534,-0.240215,0.028257,0.373450,0.529886,-0.172553,-0.004416,51.60,0
154580,102075.0,2.064449,-0.064035,-0.984886,0.199525,0.189043,-0.400122,-0.198939,-0.179360,1.895010,-0.295306,-0.003389,-2.526420,1.497146,1.805727,0.277094,0.521535,0.045203,-0.393650,-0.070260,-0.204283,-0.637231,-1.674052,0.545325,0.144882,-0.695791,-0.020314,-0.101163,-0.051533,14.97,0
174197,121814.0,2.138549,-0.873120,-1.334446,-0.965266,-0.409708,-0.486848,-0.514899,-0.057907,-0.476144,0.925710,0.829271,-0.170288,-0.915498,0.227792,-1.017833,0.794507,0.293570,-1.378713,1.118465,-0.054536,0.019041,-0.016538,0.261518,0.683928,-0.138893,-0.376241,-0.045343,-0.063591,25.00,0


In [69]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [70]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
147509   88628.0  1.720751 -0.726169 -0.808672  0.279894  0.227486  1.423688   
51130    44808.0 -7.390234 -2.362138 -1.601340  2.305168  1.322983  0.388169   
259545  159165.0 -0.704958  0.536297  1.287091 -0.470781  0.451874  0.735918   
154580  102075.0  2.064449 -0.064035 -0.984886  0.199525  0.189043 -0.400122   
174197  121814.0  2.138549 -0.873120 -1.334446 -0.965266 -0.409708 -0.486848   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9   

In [71]:
print(Y)

147509    0
51130     0
259545    0
154580    0
174197    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [72]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

In [73]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


In [74]:
print(Y.shape, Y_train.shape, Y_test.shape)

(984,) (787,) (197,)


## Model Training

In [75]:
model = LogisticRegression()

In [76]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

LogisticRegression()

In [77]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [78]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9174078780177891


In [79]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [80]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.934010152284264


In [81]:
import pickle

# Save the model to a file
with open('credit_card_log_reg.pkl', 'wb') as file:
    pickle.dump(model, file)
